# Laboratoire 1 : Extraction de primitives
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | ahmad al-taher - alta22109307                           |
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | automn 2018                                             |
| Groupe                | 2                                                       |
| Numéro du laboratoire | 00                                                      |
| Professeur            | Hervé Lombaert                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 20 sep 2018                                             |

In [26]:
import numpy as np
import math
import cv2 as cv
from scipy.misc import imread, imshow,face
import matplotlib.pyplot as plt
from skimage import img_as_ubyte
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from sklearn.datasets import load_iris
from sklearn import tree, preprocessing
import graphviz 


In [18]:
class Image:
    def __init__(self, path,label, answer):
        """
             The only construct is to build an object type image that has all the elements required to make a decision weather it is
             a smooth or spiral galaxy
             
             Args:
                 self : refers to the class
                 path : Where the image is stored
                 label : The image name 
                 answer : The final answer (smooth or spiral). This is the answer from the data set. 
                          It is used to verify after making a decision. 
                          
              Returns:
                  An Image object with the image manipulations
         """
        self.Path = path
        self.Label = label
        self.Answer = answer
        self.Pixels = np.array(cv.imread(path))
        self.Width = self.Pixels.shape[0]
        self.Hight = self.Pixels.shape[1]
        #we will always use the cropped image
        #default crop is 250
        self.crop(self.Width)
        #useful image manipulations
        self.manipulations()
        #default computations
        self.ComputeCircularity()
        self.computeBlackWhite()
        self.ComputeConvexity()
        self.ComputeBoundingRectangleToFillFactor()
        
    def manipulations(self):
        """
             This method is used to apply all the images manipulation on the image. Such as grayscale.
             Cropped image is the default image used 
             
             Args:
                 self: refers to the class
        """
        #remove noise by blurring with a Gaussian filter
        self.CroppedPixels = cv.GaussianBlur(self.CroppedPixels,(3,3), 0)
        #convert to grayscale
        self.GrayScale = cv.cvtColor(self.CroppedPixels, cv.COLOR_BGR2GRAY)
        """remove background noise
        #the result is worst with laplacian
        #laplacian = cv.Laplacian(self._GrayScale, cv.CV_16S, 3)
        #laplacian = cv.convertScaleAbs(laplacian)
        #self._GrayScale = self._GrayScale - laplacian 
        """
        self.Threshold = threshold_otsu(self.GrayScale)
        self.Binary = self.GrayScale > self.Threshold
    
    def crop(self, dimentions):
        """
             This method is used to apply a crop to the image. Since the image is squared only on parameter is required 
             
             Args:
                 dimentions: refers the final demention of the image. Such as the final image would have
                                 dimentions*dimentions. Ex: dimentions=250 the image will be 250x250
        """
        # dimention is the width and hight to crop to. Since it is a square.
        upper_width = int(self.Width/2 + dimentions/2)
        lower_width = int(self.Width/2 - dimentions/2)
        upper_height = int(self.Hight/2 + dimentions/2)
        lower_height = int(self.Hight/2 - dimentions/2)
        #new array of the image
        self.CroppedPixels = self.Pixels[ lower_width:upper_width,lower_height : upper_height]
        
    def computeBlackWhite(self):
        """
             This method is used to compute the black and white ratio.
             The formula is blacks / whites
             
             Args:
                 self: refers to the class
        """
        self.Black = 0
        self.White = 0
        self.BlackWhiteRatio = 0
        #compute the # of black and the # of whites
        for row in self.Binary :
            for pixel in row:
                if(pixel):
                    self.White += 1
                else:
                    self.Black += 1
        #compute the B/W ratio
        if self.Black > 0 and self.White >0 :
            self.BlackWhiteRatio = self.Black / self.White
        
    def ComputeCircularity(self):
        """
             This method is used to compute the circularity of the galaxy.
             The formula used is : 𝐶 = 4 ∗ 𝜋 ∗ 𝐴/𝑃2
             
             Args:
                 self: refers to the class
         """
        #example from openCV documentation
        ret,thresh = cv.threshold(self.GrayScale,self.Threshold, 255, 0)
        img, contours, hierarchy = cv.findContours(thresh,1 ,2 )
        self.cnt = contours[0]
        
        self.area = cv.contourArea(self.cnt)
        self.perimeter = cv.arcLength(self.cnt,True)
        #circularity
        self.C = 0
        if self.area >0 and self.perimeter > 0 :
            self.FormFactor = self.area/ math.pow(self.perimeter,2)
            self.C = 4* math.pi * self.FormFactor 
    
    def ComputeConvexity(self):
        """
             This method is used to compute the convexity of the galaxy.
             The formula used is : 𝐶 = P / (2W + 2H)
             where P is the perimeter
             W is the width of the bounding rectangle
             H is the height of the bounding rectangle
             
             Args:
                 self: refers to the class
         """
        x,y,w,h = cv.boundingRect(self.cnt)
        self.Convexity = self.perimeter / (2*w+2*h)

    def ComputeBoundingRectangleToFillFactor(self):
        """
             This method is used to compute the bounding rectangle to fill factor.
             The formula used is : B = A / (W*H)
             Where is A is the area of the shape
             W*H is the area of the bounding rectangle
             Args:
                 self: refers to the class
         """
        x,y,w,h = cv.boundingRect(self.cnt)
        self.B = self.area / (w*h)
                

In [39]:
def loadAllImages(dataPath , imageFolderPath):
    """
             This method is used to al the images from the data set.
             
             Args:
                 dataPath: the dataset file path
                 imageFolderPath: the image dataset folder path
         """
    dataFile = open(dataPath, 'r') # option r veut dire read
    #index is used to load a define number of images.
    index = 0
    for line in dataFile:
        texts = line.split(",")
        imageName = texts[0]
        shape = str(texts[1])

        imagePath = imageFolderPath +"\\"+str(imageName)+'.jpg'
        if index < 200 and "smooth" in line:
            index += 1
            if index % 10 <= 7:
                trainDataSet.append(Image(imagePath,imageName,shape))
            else:
                testDataSet.append(Image(imagePath,imageName,shape))
        elif index >= 200 and index < 400 and "spiral" in line:
            index += 1
            if index % 10 <= 7:
                trainDataSet.append(Image(imagePath,imageName,shape))
            else:
                testDataSet.append(Image(imagePath,imageName,shape))

    dataFile.close()
    
def  traceGraph(feature1x, feature1y, feature1Name, feature2x, feature2y, feature2Name, xlabel, ylabel):
    """
         This method is used to out a 2D graph with the selected features

         Args:
             feature1x : The feature 1 x array. It will be used for the first data set shown in the graph. 
             feature1y : The feature 1 y array. It will be used for the first data set shown in the graph.
             feature1Name : The feature 1 dataset name. It will be used for the first data set shown in the graph.
             feature2x : The feature 2 x array. It will be used for the first data set shown in the graph.
             feature2y : The feature 2yx array. It will be used for the first data set shown in the graph.
             feature2Name : The feature 2 dataset name. It will be used for the first data set shown in the graph.
             xlabel : the label shown on the x axis.
             ylabel: the label shown on the y axis:
     """
    #fig = plt.figure()
    #ax1 = fig.add_subplot()
    plt.grid(True)
    plt.scatter(feature1x, feature1y, s=10, c='b', marker="s", label=feature1Name)
    plt.scatter(feature2x, feature2y, s=10, c='r', marker="o", label=feature2Name)
    plt.ylabel(xlabel)
    plt.xlabel(ylabel)
    plt.legend(loc='upper left');
    plt.show()

def buildTree(trainArray):
    """
        This method is used to build the decision tree and how the graph associated with it.
        fit(x,y) takes two arrays:
        X, sparse or dense, of size [n_samples, n_features] holding the training samples, and an array 
                ex: [[0, 0], [1, 1]]
        Y of integer values, size [n_samples], holding the class labels 
             ex : [0, 1]
        
        Since out Y array is not numerical we are using preprocessing fron sklearn to transforme them
        Args:
            trainArray: the array containing all the features. It will be used as the X
     """
    le = preprocessing.LabelEncoder()
    le.fit(["smooth","spiral"])
    y = le.transform(["smooth","spiral"])
    
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(trainArray, y)
    dot_data = tree.export_graphviz(clf, out_file=None) 
    graph = graphviz.Source(dot_data) 
    graph.render("Galaxy") 
    

In [ ]:
#test
img = Image("C:\\Users\\ThinkPad\\Downloads\\gti770\\data\\images\\231792.jpg","232031","smooth")

#plt.imshow(img.CroppedPixels)
#plt.title("original")
#plt.show()

#plt.imshow(img.GrayScale)
#plt.title("grayscale")
#plt.show()

print("threshold: ",img.Threshold)
#plt.imshow(img.Binary)
#plt.title("binarized")
#plt.show()

print("circularity: ", img.C)

print("black ",img.Black)
print("white ", img.White)
print("B/W ratio: ", img.BlackWhiteRatio)

print("convexity: ", img.Convexity)
print("8 Bounding rectangle to fill factor: ", img.B)
#load
#sepration 70%-30% of the data set
trainDataSet = []
testDataSet = []


loadAllImages("C:\\Users\\ThinkPad\\Downloads\\gti770\\data\\csv\\galaxy\\galaxy_label_data_set.csv","C:\\Users\\ThinkPad\\Downloads\\gti770\\data\\images")
#B/W ratio
feature1x = []
feature1y = []
#circularity
feature2x = []
feature2y = []
#convexity
feature3x = []
feature3y = []
#Bounding factor
feature4x = []
feature4y = []

trainArray= []

for ig in trainDataSet:
    trainArray.append([ig.BlackWhiteRatio, ig.C, ig.Convexity, ig.B])
    if"smooth" in ig.Answer:
        feature1x.append(ig.BlackWhiteRatio)
        feature1y.append(ig.C)
        feature3x.append(ig.Convexity)
        feature3y.append(ig.B)
    else:
        feature2x.append(ig.BlackWhiteRatio)
        feature2y.append(ig.C)
        feature4x.append(ig.Convexity)
        feature4y.append(ig.B)
"""for ig in testDataSet:
    if"smooth" in ig.Answer:
        feature1x.append(ig.BlackWhiteRatio)
        feature1y.append(ig.C)
    else:
        feature2x.append(ig.BlackWhiteRatio)
        feature2y.append(ig.C)
"""

traceGraph(feature1x,feature1y,"smooth",feature2x,feature2y,"spiral", "Black/White ratio", "Circularity")
traceGraph(feature3x,feature3y,"smooth",feature4x,feature4y,"spiral", "Convexitry", "Bounding ratio")

buildTree(trainArray)

threshold:  52
circularity:  0.34290273015491973
black  171868
white  7908
B/W ratio:  21.73343449671219
convexity:  0.6178511281808218
8 Bounding rectangle to fill factor:  0.1875


### Introduction et revue de la littérature

In [ ]:
Dans le cadre de ce laboratoire, 

### Question 2

### Question 3

### Question 4

### Question 5

### Conclusion

### Bibliographie

In [214]:
https://docs.opencv.org/3.4.2/dd/d49/tutorial_py_contour_features.html
http://scikit-learn.org/stable/modules/tree.html

SyntaxError: invalid syntax (<ipython-input-214-1f74ff1184fd>, line 1)